In [1]:
import pandas as pd
import numpy as np
import geopy.distance
from Code.call_geocoding_api import CallGeocodingAPI

pd.set_option('display.max_columns', 500)

#### Define input parameters

In [2]:
INPUT_DIR = "./Data/bby_location_information.csv"
OUTPUT_DIR = "./Data/us_census_bby_location_informaton.csv"
GEOCODE_URL = "https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress"
GEOCODE_BENCHMARK_ID = 9
GEOCODE_VINTAGE_ID = 910
GEOCODE_RESPONSE_FORMAT = "json"

#### Define functions

In [3]:
def form_address(df, 
                 address_line_one_col_name = "address_line_one",
                 address_line_two_col_name = "address_line_two",
                 city_col_name = "city_name",
                 state_col_name = "state_code",
                 zip_code_col_name = "zip_code"
                ):
    """
    Function: form_address()
    Purpose : Form address that is required for input into the US Census Geocoding service.
    Input   : df (Pandas DataFrame) - data frame containing address information 
                  for each location.
            : address_line_one_col_name (str) - column in df containing first address line.
            : address_line_two_col_name (str) - column in df containing second address line.
            : city_col_name (str) - column in df containing the city for address.
            : state_col_name (str) - column in df containing state abbreviation.
            : zip_code_col_name (str) - column in df containing zip code.
    Output  : Input Pandas DataFrame with two additonal columns called 
                  address_line and api_address_input
    Author  : Zach Voller, zach.voller@bestbuy.com
    Created : January 22, 2019
    """
    # Convert any NaN entries to empty strings
    df = df.replace(np.nan, '', regex = True)
    
    # Form a single address line as required by the US Census Geocoding service
    df["address_line"] = df[address_line_one_col_name] + ' ' + df[address_line_two_col_name]
    
    # Strip any extra white space from address
    df["address_line"] = df.address_line.apply(lambda x: x.strip())
    
    # Create the full address input as required by US Census Geocoding service
    df["api_address_input"] = df[["address_line", 
                                  city_col_name,
                                  state_col_name, 
                                  zip_code_col_name
                                 ]].apply(lambda x: ', '.join(x), axis = 1)
    # Return results
    return df

#### Load BBY location data

In [4]:
bby_loc_df = pd.read_csv(INPUT_DIR, 
                         sep = ',',
                         dtype = str,
                         na_values = ''
                        )
bby_loc_df = bby_loc_df.replace(np.nan, '', regex = True)
bby_loc_df.head()

,location_id,location_name,district_id,district_name,region_id,region_name,market_id,market_name,channel_id,address_line_one,address_line_two,city_name,state_code,zip_code,country_code,time_zone_id,location_status_code
0,1783,EAST SPARKS NV,40,SACRAMENTO/RENO,31,TERRITORY 31,7,SACRAMENTO / LAS VEGAS,110,1315 SCHEELS DR,,SPARKS,NV,89434,US,05,A
1,1686,DENVER FLD SVC,99,MISCELLANEOUS,96,REGION 96,96,T96 MARKET,110,21800 E 19TH AVE,,AURORA,CO,80019,US,07,A
2,7079,HOLD/BULK DDC 79,9,DENVER 1,32,TERRITORY 32,2,DENVER / SALT LAKE CITY,110,21800 E 19TH AVE,,AURORA,CO,80019,US,07,A
3,1228,GSCC CHICAGO,99,MISCELLANEOUS,96,REGION 96,96,T96 MARKET,110,1500 REMINGTON BLVD,,BOLINGBROOK,IL,60490,US,09,A
4,9002,VDC 70 BLOOMINGTON,2,MINNEAPOLIS,32,TERRITORY 32,11,MINNEAPOLIS,110,6203(B) WEST 111TH ST,,BLOOMINGTON,MN,55438,US,09,A


In [5]:
# Define address for GeoCode API
bby_loc_df = form_address(df = bby_loc_df)
bby_loc_df.head(100)

,location_id,location_name,district_id,district_name,region_id,region_name,market_id,market_name,channel_id,address_line_one,address_line_two,city_name,state_code,zip_code,country_code,time_zone_id,location_status_code,address_line,api_address_input
0,1783,EAST SPARKS NV,40,SACRAMENTO/RENO,31,TERRITORY 31,7,SACRAMENTO / LAS VEGAS,110,1315 SCHEELS DR,,SPARKS,NV,89434,US,05,A,1315 SCHEELS DR,"1315 SCHEELS DR, SPARKS, NV, 89434"
1,1686,DENVER FLD SVC,99,MISCELLANEOUS,96,REGION 96,96,T96 MARKET,110,21800 E 19TH AVE,,AURORA,CO,80019,US,07,A,21800 E 19TH AVE,"21800 E 19TH AVE, AURORA, CO, 80019"
2,7079,HOLD/BULK DDC 79,9,DENVER 1,32,TERRITORY 32,2,DENVER / SALT LAKE CITY,110,21800 E 19TH AVE,,AURORA,CO,80019,US,07,A,21800 E 19TH AVE,"21800 E 19TH AVE, AURORA, CO, 80019"
3,1228,GSCC CHICAGO,99,MISCELLANEOUS,96,REGION 96,96,T96 MARKET,110,1500 REMINGTON BLVD,,BOLINGBROOK,IL,60490,US,09,A,1500 REMINGTON BLVD,"1500 REMINGTON BLVD, BOLINGBROOK, IL, 60490"
4,9002,VDC 70 BLOOMINGTON,2,MINNEAPOLIS,32,TERRITORY 32,11,MINNEAPOLIS,110,6203(B) WEST 111TH ST,,BLOOMINGTON,MN,55438,US,09,A,6203(B) WEST 111TH ST,"6203(B) WEST 111TH ST, BLOOMINGTON, MN, 55438"
5,1755,PAC CHINO DDC,99,MISCELLANEOUS,96,REGION 96,96,T96 MARKET,110,14310 RAMONA AVE,,CHINO,CA,91710,US,05,A,14310 RAMONA AVE,"14310 RAMONA AVE, CHINO, CA, 91710"
6,451,BEAVERTON OR,46,OREGON,31,TERRITORY 31,1,SEATTLE / PORTLAND,110,3055 SW CEDAR HILLS BLVD,,BEAVERTON,OR,97005,US,05,A,3055 SW CEDAR HILLS BLVD,"3055 SW CEDAR HILLS BLVD, BEAVERTON, OR, 97005"
7,197,KING OF PRUSSIA PA,90,EASTERN PA,35,TERRITORY 35,24,PENNSYLVANIA,110,310 GODDARD BLVD,,KING OF PRUSSIA,PA,19406,US,13,A,310 GODDARD BLVD,"310 GODDARD BLVD, KING OF PRUSSIA, PA, 19406"
8,148,GRAPEVINE TX,12,DALLAS,33,TERRITORY 33,15,DALLAS,110,1515 W STATE HWY 114,,GRAPEVINE,TX,76051,US,09,A,1515 W STATE HWY 114,"1515 W STATE HWY 114, GRAPEVINE, TX, 76051"
9,274,DAYTON MALL OH,93,COLUMBUS,34,TERRITORY 34,21,CLEVELAND / COLUMBUS,110,131 MALL WOODS DR,,DAYTON,OH,45449,US,13,A,131 MALL WOODS DR,"131 MALL WOODS DR, DAYTON, OH, 45449"


#### Available Benchmarks and Vintages
* benchmark = 9 (Public_AR_Census2010), vintage = 910 (Census2010_Census2010)
* benchmark = 8 (Public_AR_ACS2017), vintage = 8 (Current_ACS2017)
* benchmark = 4 (Public_AR_Current), vintage = 4 (Current_Current)
* benchmark = 4 (Public_AR_Current), vintage = 410 (Census2010_Current) (default)

In [2]:
geocodeAPI = CallGeocodingAPI(url = "https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress",
                              county_rurality_data_path = "./Data/County_Rural_Lookup.xlsx",
                              zcta_data_path = "./Data/Gaz_zcta_national.txt"
                             )

In [7]:
bby_loc_df['geocode_data'] = bby_loc_df['api_address_input'].apply(geocodeAPI.fetchGoecodeData)
bby_loc_df.head(10)

Finished location  10 :  6  Known,  4  Unknown
Finished location  20 :  12  Known,  8  Unknown
Finished location  30 :  17  Known,  13  Unknown
Finished location  40 :  25  Known,  15  Unknown
Finished location  50 :  32  Known,  18  Unknown
Finished location  60 :  39  Known,  21  Unknown
Finished location  70 :  44  Known,  26  Unknown
Finished location  80 :  48  Known,  32  Unknown
Finished location  90 :  55  Known,  35  Unknown
Finished location  100 :  63  Known,  37  Unknown
Finished location  110 :  67  Known,  43  Unknown
Finished location  120 :  75  Known,  45  Unknown
Finished location  130 :  81  Known,  49  Unknown
Finished location  140 :  87  Known,  53  Unknown
Finished location  150 :  94  Known,  56  Unknown
Finished location  160 :  103  Known,  57  Unknown
Finished location  170 :  110  Known,  60  Unknown
Finished location  180 :  117  Known,  63  Unknown
Finished location  190 :  126  Known,  64  Unknown
Finished location  200 :  130  Known,  70  Unknown
Finishe

Finished location  1590 :  1013  Known,  577  Unknown
Finished location  1600 :  1020  Known,  580  Unknown
Finished location  1610 :  1028  Known,  582  Unknown
Finished location  1620 :  1033  Known,  587  Unknown
Finished location  1630 :  1040  Known,  590  Unknown
Finished location  1640 :  1047  Known,  593  Unknown
Finished location  1650 :  1055  Known,  595  Unknown
Finished location  1660 :  1061  Known,  599  Unknown
Finished location  1670 :  1068  Known,  602  Unknown
Finished location  1680 :  1074  Known,  606  Unknown
Finished location  1690 :  1081  Known,  609  Unknown
Finished location  1700 :  1087  Known,  613  Unknown
Finished location  1710 :  1095  Known,  615  Unknown
Finished location  1720 :  1100  Known,  620  Unknown
Finished location  1730 :  1105  Known,  625  Unknown
Finished location  1740 :  1112  Known,  628  Unknown
Finished location  1750 :  1116  Known,  634  Unknown
Finished location  1760 :  1119  Known,  641  Unknown
Finished location  1770 :  1

,location_id,location_name,district_id,district_name,region_id,region_name,market_id,market_name,channel_id,address_line_one,address_line_two,city_name,state_code,zip_code,country_code,time_zone_id,location_status_code,address_line,api_address_input,geocode_data
0,1783,EAST SPARKS NV,40,SACRAMENTO/RENO,31,TERRITORY 31,7,SACRAMENTO / LAS VEGAS,110,1315 SCHEELS DR,,SPARKS,NV,89434,US,05,A,1315 SCHEELS DR,"1315 SCHEELS DR, SPARKS, NV, 89434",{}
1,1686,DENVER FLD SVC,99,MISCELLANEOUS,96,REGION 96,96,T96 MARKET,110,21800 E 19TH AVE,,AURORA,CO,80019,US,07,A,21800 E 19TH AVE,"21800 E 19TH AVE, AURORA, CO, 80019",{}
2,7079,HOLD/BULK DDC 79,9,DENVER 1,32,TERRITORY 32,2,DENVER / SALT LAKE CITY,110,21800 E 19TH AVE,,AURORA,CO,80019,US,07,A,21800 E 19TH AVE,"21800 E 19TH AVE, AURORA, CO, 80019",{}
3,1228,GSCC CHICAGO,99,MISCELLANEOUS,96,REGION 96,96,T96 MARKET,110,1500 REMINGTON BLVD,,BOLINGBROOK,IL,60490,US,09,A,1500 REMINGTON BLVD,"1500 REMINGTON BLVD, BOLINGBROOK, IL, 60490","{'latitude': 41.6624, 'longitude': -88.12594, ..."
4,9002,VDC 70 BLOOMINGTON,2,MINNEAPOLIS,32,TERRITORY 32,11,MINNEAPOLIS,110,6203(B) WEST 111TH ST,,BLOOMINGTON,MN,55438,US,09,A,6203(B) WEST 111TH ST,"6203(B) WEST 111TH ST, BLOOMINGTON, MN, 55438",{}
5,1755,PAC CHINO DDC,99,MISCELLANEOUS,96,REGION 96,96,T96 MARKET,110,14310 RAMONA AVE,,CHINO,CA,91710,US,05,A,14310 RAMONA AVE,"14310 RAMONA AVE, CHINO, CA, 91710","{'latitude': 33.99343, 'longitude': -117.70642..."
6,451,BEAVERTON OR,46,OREGON,31,TERRITORY 31,1,SEATTLE / PORTLAND,110,3055 SW CEDAR HILLS BLVD,,BEAVERTON,OR,97005,US,05,A,3055 SW CEDAR HILLS BLVD,"3055 SW CEDAR HILLS BLVD, BEAVERTON, OR, 97005","{'latitude': 45.48887, 'longitude': -122.81083..."
7,197,KING OF PRUSSIA PA,90,EASTERN PA,35,TERRITORY 35,24,PENNSYLVANIA,110,310 GODDARD BLVD,,KING OF PRUSSIA,PA,19406,US,13,A,310 GODDARD BLVD,"310 GODDARD BLVD, KING OF PRUSSIA, PA, 19406","{'latitude': 40.090137, 'longitude': -75.40741..."
8,148,GRAPEVINE TX,12,DALLAS,33,TERRITORY 33,15,DALLAS,110,1515 W STATE HWY 114,,GRAPEVINE,TX,76051,US,09,A,1515 W STATE HWY 114,"1515 W STATE HWY 114, GRAPEVINE, TX, 76051","{'latitude': 32.930344, 'longitude': -97.09527..."
9,274,DAYTON MALL OH,93,COLUMBUS,34,TERRITORY 34,21,CLEVELAND / COLUMBUS,110,131 MALL WOODS DR,,DAYTON,OH,45449,US,13,A,131 MALL WOODS DR,"131 MALL WOODS DR, DAYTON, OH, 45449","{'latitude': 39.640278, 'longitude': -84.22775..."


In [8]:
bby_loc_df.to_csv(OUTPUT_DIR, sep = '\t')

In [1]:
geocodeAPI.known_addresses

NameError: name 'geocodeAPI' is not defined

In [ ]:
bby_loc_df = 
unknown_address_df = bby_loc_df.loc[]

#### Scratch work

In [9]:
test_df = pd.read_csv("./Data/Gaz_zcta_national.txt", sep = '\t', names = ['zip_code', 'population', 'housing_unit_cost', 'land_area', 'water_area', 'land_area_sqr_miles', 'water_area_sqr_miles', 'latitude', 'longitude'], skiprows = 1, dtype = {'zip_code':str})
test_df.head(10)

,zip_code,population,housing_unit_cost,land_area,water_area,land_area_sqr_miles,water_area_sqr_miles,latitude,longitude
0,00601,18570,7744,166659789,799296,64.348,0.309,18.180555,-66.749961
1,00602,41520,18073,79288158,4446273,30.613,1.717,18.362268,-67.176130
2,00603,54689,25653,81880442,183425,31.614,0.071,18.455183,-67.119887
3,00606,6615,2877,109580061,12487,42.309,0.005,18.158345,-66.932911
4,00610,29016,12618,93021467,4172001,35.916,1.611,18.290955,-67.125868
5,00612,67010,30992,175106243,9809163,67.609,3.787,18.402239,-66.711400
6,00616,11017,4896,29870473,149147,11.533,0.058,18.420412,-66.671979
7,00617,24597,10594,39347158,3987969,15.192,1.540,18.445147,-66.559696
8,00622,7853,8714,75077028,1694917,28.987,0.654,17.991245,-67.153993
9,00623,43061,21426,98367847,1633540,37.980,0.631,18.083435,-67.153877


In [12]:
test2_df = test_df.set_index('zip_code').to_dict()['latitude']
test2_df['00601']

18.180555

In [3]:
dest_df = pd.read_csv("./Data/order_shipment_addresses.csv", 
                      sep = ',',
                      dtype = str
                     )
dest_df.head(10)

,CUSTOMER_PO_NO,ENTERPRISE_KEY,DIVISION,ORDER_DATE,ORDER_TYPE,ENTRY_TYPE,SELLER_ORGANIZATION_CODE,SHIP_TO_ADDRESS_LINE1,SHIP_TO_ADDRESS_LINE2,SHIP_TO_ADDRESS_LINE3,SHIP_TO_CITY,SHIP_TO_STATE,SHIP_TO_ZIP_CODE,SHIP_TO_COUNTRY
0,1118315678980,RETAIL,NaN,2018-11-11 00:11:55.000000,NaN,POS,BBY_852,94-1035 MAHINAHINA ST,NaN,NaN,WAIPAHU,HI,967974843,US
1,1118315679166,RETAIL,NaN,2018-11-11 00:18:05.000000,NaN,POS,BBY_763,6977 KALANIANAOLE HWY,NaN,NaN,HONOLULU,HI,968252009,US
2,1118315679193,RETAIL,NaN,2018-11-11 00:21:23.000000,NaN,POS,BBY_358,191 VERDE RIDGE CT,NaN,NaN,HENDERSON,NV,890122452,US
3,1118315679242,RETAIL,NaN,2018-11-11 00:37:40.000000,NaN,POS,BBY_852,801 2ND ST,NaN,NaN,PEARL CITY,HI,967823343,US
4,1118315679258,RETAIL,NaN,2018-11-11 00:51:10.000000,NaN,POS,BBY_763,625 MALUNIU AVE,NaN,NaN,KAILUA,HI,967342155,US
5,1118315679333,RETAIL,NaN,2018-11-11 01:03:34.000000,NaN,CoreBlue,BBY_852,2101 WILSON ST,NaN,NaN,HONOLULU,HI,96819,US
6,1118315679701,RETAIL,NaN,2018-11-11 04:21:45.000000,NaN,NaN,BBY_335,5111 HILTON CT,NaN,NaN,LOUISVILLE,KY,40218-4125,US
7,1118315679702,RETAIL,NaN,2018-11-11 04:21:46.000000,NaN,NaN,BBY_335,5111 HILTON CT,NaN,NaN,LOUISVILLE,KY,40218-4125,US
8,1118315680088,RETAIL,NaN,2018-11-11 07:30:51.000000,NaN,NaN,BBY_1324,385 S BEECH ST,NaN,NaN,MANCHESTER,NH,03103-6505,US
9,1118315680089,RETAIL,NaN,2018-11-11 07:30:52.000000,NaN,NaN,BBY_1324,385 S BEECH ST,NaN,NaN,MANCHESTER,NH,03103-6505,US


In [4]:
dest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7916314 entries, 0 to 7916313
Data columns (total 14 columns):
CUSTOMER_PO_NO              object
ENTERPRISE_KEY              object
DIVISION                    object
ORDER_DATE                  object
ORDER_TYPE                  object
ENTRY_TYPE                  object
SELLER_ORGANIZATION_CODE    object
SHIP_TO_ADDRESS_LINE1       object
SHIP_TO_ADDRESS_LINE2       object
SHIP_TO_ADDRESS_LINE3       object
SHIP_TO_CITY                object
SHIP_TO_STATE               object
SHIP_TO_ZIP_CODE            object
SHIP_TO_COUNTRY             object
dtypes: object(14)
memory usage: 845.6+ MB


In [5]:
dest_df = geocodeAPI.parseZipcodes(df = dest_df, zip_code_col_name = 'SHIP_TO_ZIP_CODE')
dest_df['latitude'] = dest_df.parsed_zip_codes.apply(lambda x: geocodeAPI.zipcodes_to_latitude.get(x))
dest_df['longitude'] = dest_df.parsed_zip_codes.apply(lambda x: geocodeAPI.zipcodes_to_longitude.get(x))
dest_df.head(10)

,CUSTOMER_PO_NO,ENTERPRISE_KEY,DIVISION,ORDER_DATE,ORDER_TYPE,ENTRY_TYPE,SELLER_ORGANIZATION_CODE,SHIP_TO_ADDRESS_LINE1,SHIP_TO_ADDRESS_LINE2,SHIP_TO_ADDRESS_LINE3,SHIP_TO_CITY,SHIP_TO_STATE,SHIP_TO_ZIP_CODE,SHIP_TO_COUNTRY,parsed_zip_codes,latitude,longitude
0,1118315678980,RETAIL,NaN,2018-11-11 00:11:55.000000,NaN,POS,BBY_852,94-1035 MAHINAHINA ST,NaN,NaN,WAIPAHU,HI,967974843,US,96797,21.417469,-157.983838
1,1118315679166,RETAIL,NaN,2018-11-11 00:18:05.000000,NaN,POS,BBY_763,6977 KALANIANAOLE HWY,NaN,NaN,HONOLULU,HI,968252009,US,96825,21.294518,-157.688748
2,1118315679193,RETAIL,NaN,2018-11-11 00:21:23.000000,NaN,POS,BBY_358,191 VERDE RIDGE CT,NaN,NaN,HENDERSON,NV,890122452,US,89012,36.011893,-115.043260
3,1118315679242,RETAIL,NaN,2018-11-11 00:37:40.000000,NaN,POS,BBY_852,801 2ND ST,NaN,NaN,PEARL CITY,HI,967823343,US,96782,21.417374,-157.936867
4,1118315679258,RETAIL,NaN,2018-11-11 00:51:10.000000,NaN,POS,BBY_763,625 MALUNIU AVE,NaN,NaN,KAILUA,HI,967342155,US,96734,21.395084,-157.758188
5,1118315679333,RETAIL,NaN,2018-11-11 01:03:34.000000,NaN,CoreBlue,BBY_852,2101 WILSON ST,NaN,NaN,HONOLULU,HI,96819,US,96819,21.347927,-157.876274
6,1118315679701,RETAIL,NaN,2018-11-11 04:21:45.000000,NaN,NaN,BBY_335,5111 HILTON CT,NaN,NaN,LOUISVILLE,KY,40218-4125,US,40218,38.189321,-85.654311
7,1118315679702,RETAIL,NaN,2018-11-11 04:21:46.000000,NaN,NaN,BBY_335,5111 HILTON CT,NaN,NaN,LOUISVILLE,KY,40218-4125,US,40218,38.189321,-85.654311
8,1118315680088,RETAIL,NaN,2018-11-11 07:30:51.000000,NaN,NaN,BBY_1324,385 S BEECH ST,NaN,NaN,MANCHESTER,NH,03103-6505,US,03103,42.950538,-71.446547
9,1118315680089,RETAIL,NaN,2018-11-11 07:30:52.000000,NaN,NaN,BBY_1324,385 S BEECH ST,NaN,NaN,MANCHESTER,NH,03103-6505,US,03103,42.950538,-71.446547


In [6]:
data_df = pd.read_csv("./Data/data_OM_inventory_and_date_add_1-26-2019.txt", 
                      sep = '\t',
                      dtype = str
                     )
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day
0,2018111614203598721255,2018-11-16T20:20:36.833Z,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313
1,2018111413520911168760,2018-11-14T19:52:11.261Z,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17
2,2018112005325987886463,2018-11-20T11:33:02.200Z,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9
3,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14
4,2018111214335523525015,2018-11-12T20:33:56.473Z,2018-11-13 06:00:00 UTC,6285506,2018-11-12T20:33:54Z,BBY01-805580375984,NIB,66434,BESTBUY,1,7,BBY_224,STORE,2068864198041,3,1921958822953,1751785385,20020404405,80081617623,2002,5005101,SDS,57600,57600,Shipped and Invoiced,2018-11-13,2018-11-13,BBY_224,UPS,1ZF1Y4700326336569,03,N,UPSG,ground,ground,2018-11-12,1,Monday,46,2.47300004959106,51,16
5,2018111818365470489848,2018-11-19T00:39:07.847Z,2018-11-21 06:00:00 UTC,5732908,2018-11-19T00:36:54Z,1118322371156,NIB,80504,RETAIL,1,7,BBY_209,STORE,2069364808242,14,1921958822953,2252295495,20020404405,80081617623,2002,5005101,MS,59400,58500,Shipped and Invoiced,2018-11-21,NaN,BBY_209,UPS,1ZF2368X0385451981,03,N,UPSG,ground,ground,2018-11-19,2,Monday,47,133.84700012207,408,38
6,2018111717335322932016,2018-11-17T23:43:10.562Z,2018-11-19 06:00:00 UTC,4600801,2018-11-17T23:33:53Z,1118321279225,NIB,84403,RETAIL,1,7,BBY_9008,DC,5911534923489,1626,5765876468859,500510110,20020404405,80081617623,4004,10010202,DC,54000,54000,Shipped and Invoiced,2018-11-19,2018-12-19,BBY_725,UPS,1Z4260541309887034,13,Y,UPS1P,air,ground,2018-11-17,2,Saturday,46,557.56200003624,954194,1536
7,2018111412232494071020,2018-11-14T18:35:41.220Z,2018-12-04 18:35:40 UTC,6139651,2018-11-14T18:23:24Z,1118318910234,NIB,77713,RETAIL,1,1,BBY_9024,DC,2068373051465,12,1921958822953,1251275275,20020404405,80081617623,6006,15015303,DC,44100,46800,Shipped and Invoiced,NaN,NaN,BBY_9024,UPS,1ZW495471384330232,13,N,UPS1P,air,ground,2018-11-14,NaN,Wednesday,46,737.22000002861,135473,193
8,2018112009430317902193,2018-11-20T15:43:05.627Z,2018-11-23 06:00:00 UTC,5873917,2018-11-20T15:43:02Z,BBY01-805582618644,NIB,94510,BESTBUY,2,7,BBY_142,STORE,2068864047855,36,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,54000,53100,

In [7]:
bby_loc_df = pd.read_csv("./Data/bby_location_data.txt", 
                         sep = '\t',
                         dtype = str
                        )
bby_loc_df.head(10)

,SHP_NODE_ID,LATIT_VAL,LONGIT_VAL,ADDR_LN1_TXT,ADDR_LN2_TXT,ADDR_CITY_NM,ADDR_STATE_CDE,ADDR_ZIP_CDE,ADDR_CTRY_CDE
0,BBY_10,45.03655600,-93.02598600,1795 County Rd D E,NaN,Maplewood,MN,55109,US
1,BBY_1000,44.85466000,-93.24565000,340 W Market,NaN,Bloomington,MN,55425,US
2,BBY_1002,33.43072900,-111.89966000,1900 E Rio Salado Pkwy,NaN,Tempe,AZ,85281,US
3,BBY_1003,38.29697000,-76.51201600,45235 Worth Ave,NaN,California,MD,20619,US
4,BBY_1004,33.60543800,-86.64266200,5072 Pinnacle Sq,NaN,Birmingham,AL,35235,US
5,BBY_1008,38.36764900,-121.96328000,1621 E Monte Vista Ave,NaN,Vacaville,CA,95688,US
6,BBY_1009,33.76035300,-118.11414000,6310 E Pacific Coast Hwy,NaN,Long Beach,CA,90803,US
7,BBY_101,33.59059900,-117.67481400,25422 El Paseo,NaN,Mission Viejo,CA,92691,US
8,BBY_1010,38.32717500,-85.76461000,1025 Veterans Pkwy,NaN,Clarksville,IN,47129,US
9,BBY_1011,33.90758100,-118.39295000,740 S Sepulveda Blvd,NaN,El Segundo,CA,90245,US


In [8]:
ship_from_loc_df = bby_loc_df[['SHP_NODE_ID', 'LATIT_VAL', 'LONGIT_VAL']]
ship_from_loc_df.columns = ['ship_from_loc_id', 'ship_from_loc_lat', 'ship_from_loc_long']
ship_from_loc_df.head(10)

,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long
0,BBY_10,45.03655600,-93.02598600
1,BBY_1000,44.85466000,-93.24565000
2,BBY_1002,33.43072900,-111.89966000
3,BBY_1003,38.29697000,-76.51201600
4,BBY_1004,33.60543800,-86.64266200
5,BBY_1008,38.36764900,-121.96328000
6,BBY_1009,33.76035300,-118.11414000
7,BBY_101,33.59059900,-117.67481400
8,BBY_1010,38.32717500,-85.76461000
9,BBY_1011,33.90758100,-118.39295000


In [9]:
data_df = data_df.merge(ship_from_loc_df,
                        left_on = 'SHIP_NODE',
                        right_on = 'ship_from_loc_id',
                        how = 'left'
                       )
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long
0,2018111614203598721255,2018-11-16T20:20:36.833Z,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313,BBY_2600,38.06069100,-85.72090400
1,2018111413520911168760,2018-11-14T19:52:11.261Z,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17,BBY_1458,29.04414900,-95.45613100
2,2018112005325987886463,2018-11-20T11:33:02.200Z,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9,BBY_1475,30.47299000,-81.64357000
3,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300
4,2018111214335523525015,2018-11-12T20:33:56.473Z,2018-11-13 06:00:00 UTC,6285506,2018-11-12T20:33:54Z,BBY01-805580375984,NIB,66434,BESTBUY,1,7,BBY_224,STORE,2068864198041,3,1921958822953,1751785385,20020404405,80081617623,2002,5005101,SDS,57600,57600,Shipped and Invoiced,2018-11-13,2018-11-13,BBY_224,UPS,1ZF1Y4700326336569,03,N,UPSG,ground,ground,2018-11-12,1,Monday,46,2.47300004959106,51,16,BBY_224,37.67517100,-97.42003600
5,2018111818365470489848,2018-11-19T00:39:07.847Z,2018-11-21 06:00:00 UTC,5732908,2018-11-19T00:36:54Z,1118322371156,NIB,80504,RETAIL,1,7,BBY_209,STORE,2069364808242,14,1921958822953,2252295495,20020404405,80081617623,2002,5005101,MS,59400,58500,Shipped and Invoiced,2018-11-21,NaN,BBY_209,UPS,1ZF2368X0385451981,03,N,UPSG,ground,ground,2018-11-19,2,Monday,47,133.84700012207,408,38,BBY_209,39.86720700,-105.05496000
6,2018111717335322932016,2018-11-17T23:43:10.562Z,2018-11-19 06:00:00 UTC,4600801,2018-11-17T23:33:53Z,1118321279225,NIB,84403,RETAIL,1,7,BBY_9008,DC,5911534923489,1626,5765876468859,500510110,20020404405,80081617623,4004,10010202,DC,54000,54000,Shipped and Invoiced,2018-11-19,2018-12-19,BBY_725,UPS,1Z4260541309887034,13,Y,UPS1P,air,ground,2018-11-17,2,Saturday,46,557.56200003624,954194,1536,BBY_725,36.82200000,-119.40300000
7,2018111412232494071020,2018-11-14T18:35:41.220Z,2018-12-04 18:35:40 UTC,6139651,2018-11-14T18:23:24Z,1118318910234,NIB,77713,RETAIL,1,1,BBY_9024,DC,2068373051465,12,1921958822953,1251275275,20020404405,80081617623,6006,15015303,DC,44100,46800,Shipped and Invoiced,NaN,NaN,BBY_9024,UPS,1ZW495471384330232,13,N,UPS1P,air,ground,2018-11-14,NaN,We

In [10]:
ship_to_loc_df = dest_df[['CUSTOMER_PO_NO', 'parsed_zip_codes', 'latitude', 'longitude']]
ship_to_loc_df.columns = ['dest_order_number', 'dest_zip_code', 'dest_lat', 'dest_long']
ship_to_loc_df.head(10)

,dest_order_number,dest_zip_code,dest_lat,dest_long
0,1118315678980,96797,21.417469,-157.983838
1,1118315679166,96825,21.294518,-157.688748
2,1118315679193,89012,36.011893,-115.043260
3,1118315679242,96782,21.417374,-157.936867
4,1118315679258,96734,21.395084,-157.758188
5,1118315679333,96819,21.347927,-157.876274
6,1118315679701,40218,38.189321,-85.654311
7,1118315679702,40218,38.189321,-85.654311
8,1118315680088,03103,42.950538,-71.446547
9,1118315680089,03103,42.950538,-71.446547


In [15]:
ship_to_loc_df.loc[ship_to_loc_df.dest_order_number.str.contains("BBY")]

,dest_order_number,dest_zip_code,dest_lat,dest_long
860042,BBY01-805561463244,61103,42.340261,-89.086492
860043,BBY01-805561534908,92308,34.425262,-117.161732
860044,BBY01-805561534908,92307,34.571446,-117.133076
860045,BBY01-805561543674,35806,34.762538,-86.685033
860046,BBY01-805561543674,35803,34.563686,-86.519304
860047,BBY01-805561562576,29607,34.811726,-82.331435
860048,BBY01-805561565111,60619,41.743690,-87.605526
860049,BBY01-805561566827,96701,21.406053,-157.884952
860050,BBY01-805561566827,96701,21.406053,-157.884952
860051,BBY01-805561579792,46034,40.127927,-86.046784


In [11]:
data_df = data_df.merge(ship_to_loc_df,
                        left_on = 'order_no',
                        right_on = 'dest_order_number',
                        how = 'left'
                       )
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long,dest_order_number,dest_zip_code,dest_lat,dest_long
0,2018111614203598721255,2018-11-16T20:20:36.833Z,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313,BBY_2600,38.06069100,-85.72090400,NaN,NaN,NaN,NaN
1,2018111413520911168760,2018-11-14T19:52:11.261Z,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17,BBY_1458,29.04414900,-95.45613100,NaN,NaN,NaN,NaN
2,2018112005325987886463,2018-11-20T11:33:02.200Z,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9,BBY_1475,30.47299000,-81.64357000,BBY01-805582422630,32507,30.339837,-87.373783
3,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,02324,41.972387,-70.978778
4,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,30101,34.034515,-84.707349
5,2018111214335523525015,2018-11-12T20:33:56.473Z,2018-11-13 06:00:00 UTC,6285506,2018-11-12T20:33:54Z,BBY01-805580375984,NIB,66434,BESTBUY,1,7,BBY_224,STORE,2068864198041,3,1921958822953,1751785385,20020404405,80081617623,2002,5005101,SDS,57600,57600,Shipped and Invoiced,2018-11-13,2018-11-13,BBY_224,UPS,1ZF1Y4700326336569,03,N,UPSG,ground,ground,2018-11-12,1,Monday,46,2.47300004959106,51,16,BBY_224,37.67517100,-97.42003600,BBY01-805580375984,66434,39.861691,-95.549365
6,2018111818365470489848,2018-11-19T00:39:07.847Z,2018-11-21 06:00:00 UTC,5732908,2018-11-19T00:36:54Z,1118322371156,NIB,80504,RETAIL,1,7,BBY_209,STORE,2069364808242,14,1921958822953,2252295495,20020404405,80081617623,2002,5005101,MS,59400,58500,Shipped and Invoiced,2018-11-21,NaN,BBY_209,UPS,1ZF2368X0385451981,03,N,UPSG,ground,ground,2018-11-19,2,Monday,47,133.84700012207,408,38,BBY_209,39.86720700,-105.05496000,1118322371156,80504,40.163388,-105.030087
7,2018111717335322932016,2018-11-1

In [12]:
rec_ship_from_loc_df = bby_loc_df[['SHP_NODE_ID', 'LATIT_VAL', 'LONGIT_VAL']]
rec_ship_from_loc_df.columns = ['rec_ship_from_loc_id', 'rec_ship_from_loc_lat', 'rec_ship_from_loc_long']
rec_ship_from_loc_df.head(10)

,rec_ship_from_loc_id,rec_ship_from_loc_lat,rec_ship_from_loc_long
0,BBY_10,45.03655600,-93.02598600
1,BBY_1000,44.85466000,-93.24565000
2,BBY_1002,33.43072900,-111.89966000
3,BBY_1003,38.29697000,-76.51201600
4,BBY_1004,33.60543800,-86.64266200
5,BBY_1008,38.36764900,-121.96328000
6,BBY_1009,33.76035300,-118.11414000
7,BBY_101,33.59059900,-117.67481400
8,BBY_1010,38.32717500,-85.76461000
9,BBY_1011,33.90758100,-118.39295000


In [13]:
data_df = data_df.merge(rec_ship_from_loc_df,
                        left_on = 'loc_id',
                        right_on = 'rec_ship_from_loc_id',
                        how = 'left'
                       )
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long,dest_order_number,dest_zip_code,dest_lat,dest_long,rec_ship_from_loc_id,rec_ship_from_loc_lat,rec_ship_from_loc_long
0,2018111614203598721255,2018-11-16T20:20:36.833Z,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313,BBY_2600,38.06069100,-85.72090400,NaN,NaN,NaN,NaN,BBY_2600,38.06069100,-85.72090400
1,2018111413520911168760,2018-11-14T19:52:11.261Z,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17,BBY_1458,29.04414900,-95.45613100,NaN,NaN,NaN,NaN,BBY_1458,29.04414900,-95.45613100
2,2018112005325987886463,2018-11-20T11:33:02.200Z,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9,BBY_1475,30.47299000,-81.64357000,BBY01-805582422630,32507,30.339837,-87.373783,BBY_1475,30.47299000,-81.64357000
3,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,02324,41.972387,-70.978778,BBY_519,33.72742100,-84.75784300
4,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,30101,34.034515,-84.707349,BBY_519,33.72742100,-84.75784300
5,2018111214335523525015,2018-11-12T20:33:56.473Z,2018-11-13 06:00:00 UTC,6285506,2018-11-12T20:33:54Z,BBY01-805580375984,NIB,66434,BESTBUY,1,7,BBY_224,STORE,2068864198041,3,1921958822953,1751785385,20020404405,80081617623,2002,5005101,SDS,57600,57600,Shipped and Invoiced,2018-11-13,2018-11-13,BBY_224,UPS,1ZF1Y4700326336569,03,N,UPSG,ground,ground,2018-11-12,1,Monday,46,2.47300004959106,51,16,BBY_224,37.67517100,-97.42003600,BBY01-805580375984,66434,39.861691,-95.549365,BBY_224,37.67517100,-97.42003600
6,2018111818365470489848,2018-11-19T00:39:07.847Z,2018-11-21 06:00:00 UTC,5732908,2018-11-19T00:36:54Z,1118322371156,NIB,80504,RETAIL,1,7,BBY_209,STORE,2069364808242,14,1921958822953,2252295495,20020404405,80081617623,2002,500

In [14]:
data_df['ship_from_loc_coords'] = data_df[['ship_from_loc_lat', 'ship_from_loc_long']].apply(tuple, axis = 1)
data_df['rec_ship_from_loc_coords'] = data_df[['rec_ship_from_loc_lat', 'rec_ship_from_loc_long']].apply(tuple, axis = 1)
data_df['dest_coords'] = data_df[['dest_lat', 'dest_long']].apply(tuple, axis = 1)
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long,dest_order_number,dest_zip_code,dest_lat,dest_long,rec_ship_from_loc_id,rec_ship_from_loc_lat,rec_ship_from_loc_long,ship_from_loc_coords,rec_ship_from_loc_coords,dest_coords
0,2018111614203598721255,2018-11-16T20:20:36.833Z,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313,BBY_2600,38.06069100,-85.72090400,NaN,NaN,NaN,NaN,BBY_2600,38.06069100,-85.72090400,"(38.06069100, -85.72090400)","(38.06069100, -85.72090400)","(nan, nan)"
1,2018111413520911168760,2018-11-14T19:52:11.261Z,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17,BBY_1458,29.04414900,-95.45613100,NaN,NaN,NaN,NaN,BBY_1458,29.04414900,-95.45613100,"(29.04414900, -95.45613100)","(29.04414900, -95.45613100)","(nan, nan)"
2,2018112005325987886463,2018-11-20T11:33:02.200Z,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9,BBY_1475,30.47299000,-81.64357000,BBY01-805582422630,32507,30.339837,-87.373783,BBY_1475,30.47299000,-81.64357000,"(30.47299000, -81.64357000)","(30.47299000, -81.64357000)","(30.339837, -87.37378299999999)"
3,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,02324,41.972387,-70.978778,BBY_519,33.72742100,-84.75784300,"(33.72742100, -84.75784300)","(33.72742100, -84.75784300)","(41.972387, -70.978778)"
4,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,30101,34.034515,-84.707349,BBY_519,33.72742100,-84.75784300,"(33.72742100, -84.75784300)","(33.72742100, -84.75784300)","(34.034515, -84.707349)"
5,2018111214335523525015,2018-11-12T20:33:56.473Z,2018-11-13 06:00:00 UTC,6285506,2018-11-12T20:33:54Z,BBY01-805580375984,NIB,66434,BESTBUY,1,7,BBY_224,STORE,2068864198041,3,1921958822953,1751785385,20020404405,80081617623,2002,5005101,SDS,57600,57600,Shipped and 

In [23]:
#data_df['actual_distance_shipped'] = data_df[['ship_from_loc_coords', 'dest_coords']].apply(lambda x: getDistance(x[0], x[1]), axis = 1)
data_df['recommended_distance_shipped'] = data_df[['rec_ship_from_loc_coords', 'dest_coords']].apply(lambda x: getDistance(x[0], x[1]), axis = 1)
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long,dest_order_number,dest_zip_code,dest_lat,dest_long,rec_ship_from_loc_id,rec_ship_from_loc_lat,rec_ship_from_loc_long,ship_from_loc_coords,rec_ship_from_loc_coords,dest_coords,actual_distance_shipped,recommended_distance_shipped
0,2018111614203598721255,2018-11-16T20:20:36.833Z,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313,BBY_2600,38.06069100,-85.72090400,NaN,NaN,NaN,NaN,BBY_2600,38.06069100,-85.72090400,"(38.06069100, -85.72090400)","(38.06069100, -85.72090400)","(nan, nan)",NaN,NaN
1,2018111413520911168760,2018-11-14T19:52:11.261Z,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17,BBY_1458,29.04414900,-95.45613100,NaN,NaN,NaN,NaN,BBY_1458,29.04414900,-95.45613100,"(29.04414900, -95.45613100)","(29.04414900, -95.45613100)","(nan, nan)",NaN,NaN
2,2018112005325987886463,2018-11-20T11:33:02.200Z,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9,BBY_1475,30.47299000,-81.64357000,BBY01-805582422630,32507,30.339837,-87.373783,BBY_1475,30.47299000,-81.64357000,"(30.47299000, -81.64357000)","(30.47299000, -81.64357000)","(30.339837, -87.37378299999999)",342.225758,342.225758
3,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,02324,41.972387,-70.978778,BBY_519,33.72742100,-84.75784300,"(33.72742100, -84.75784300)","(33.72742100, -84.75784300)","(41.972387, -70.978778)",942.012832,942.012832
4,2018112015160759573686,2018-11-20T21:16:11.278Z,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,30101,34.034515,-84.707349,BBY_519,33.72742100,-84.75784300,"(33.72742100, -84.75784300)","(33.72742100, -84.75784300)","(34.034515, -84.707349)",21.363836,21.363836
5,2018111214335523525015,2018-11-12T20:33:56.473Z,2018-11-13 06:00:00 UTC,6285506,2018-11-12T20:33:54Z,BBY01-805580375984,NIB,66434

In [21]:
def getDistance(ship_coords, dest_coords):
    """

    """
    coords1 = float(ship_coords[0]), float(ship_coords[1])
    coords2 = float(dest_coords[0]), float(dest_coords[1])
    
    if math.isnan(coords1[0]) or math.isnan(coords2[0]):
        return None
    else:
        return geopy.distance.distance(coords1, coords2).miles

In [20]:
import math

In [29]:
data_df['eventTime'] = pd.to_datetime(data_df['eventTime'])
data_df['eventTimeDate'] = data_df['eventTime'].dt.strftime("%Y-%m-%d")
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long,dest_order_number,dest_zip_code,dest_lat,dest_long,rec_ship_from_loc_id,rec_ship_from_loc_lat,rec_ship_from_loc_long,ship_from_loc_coords,rec_ship_from_loc_coords,dest_coords,actual_distance_shipped,recommended_distance_shipped,eventTimeDate
0,2018111614203598721255,2018-11-16 20:20:36.833,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313,BBY_2600,38.06069100,-85.72090400,NaN,NaN,NaN,NaN,BBY_2600,38.06069100,-85.72090400,"(38.06069100, -85.72090400)","(38.06069100, -85.72090400)","(nan, nan)",NaN,NaN,2018-11-16
1,2018111413520911168760,2018-11-14 19:52:11.261,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17,BBY_1458,29.04414900,-95.45613100,NaN,NaN,NaN,NaN,BBY_1458,29.04414900,-95.45613100,"(29.04414900, -95.45613100)","(29.04414900, -95.45613100)","(nan, nan)",NaN,NaN,2018-11-14
2,2018112005325987886463,2018-11-20 11:33:02.200,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9,BBY_1475,30.47299000,-81.64357000,BBY01-805582422630,32507,30.339837,-87.373783,BBY_1475,30.47299000,-81.64357000,"(30.47299000, -81.64357000)","(30.47299000, -81.64357000)","(30.339837, -87.37378299999999)",342.225758,342.225758,2018-11-20
3,2018112015160759573686,2018-11-20 21:16:11.278,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,02324,41.972387,-70.978778,BBY_519,33.72742100,-84.75784300,"(33.72742100, -84.75784300)","(33.72742100, -84.75784300)","(41.972387, -70.978778)",942.012832,942.012832,2018-11-20
4,2018112015160759573686,2018-11-20 21:16:11.278,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,30101,34.034515,-84.707349,BBY_519,33.72742100,-84.75784300,"(33.72742100, -84.75784300)","(33.72742100, -84.75784300)","(34.034515, -84.707349)",21.363836,21.363836,2018-11-20
5,2018111214335523525015,2018-11-12 20:33:56.473,2018-11-13 06:00:0

In [30]:
actual_tp = pd.read_csv("./Data/data_actual_inventory_throughput_by_date_1-27-2019.txt", 
                         sep = '\t',
                         dtype = str
                        )
actual_tp.columns = ['actual_eventDate', 'actual_SHIP_NODE', 'act_loc_qty_day', 'act_loc_order_qty_day']
actual_tp.head(10)

,actual_eventDate,actual_SHIP_NODE,act_loc_qty_day,act_loc_order_qty_day
0,2018-11-11,BBY_10,50,11
1,2018-11-11,BBY_1002,50,14
2,2018-11-11,BBY_1003,20,10
3,2018-11-11,BBY_1004,57,9
4,2018-11-11,BBY_1008,13,12
5,2018-11-11,BBY_101,54,8
6,2018-11-11,BBY_1010,34,22
7,2018-11-11,BBY_1011,13,6
8,2018-11-11,BBY_1012,43,17
9,2018-11-11,BBY_1013,129,21


In [31]:
data_df = data_df.merge(actual_tp,
                        left_on = ['SHIP_NODE', 'eventTimeDate'],
                        right_on = ['actual_SHIP_NODE', 'actual_eventDate'],
                        how = 'left'
                       )
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long,dest_order_number,dest_zip_code,dest_lat,dest_long,rec_ship_from_loc_id,rec_ship_from_loc_lat,rec_ship_from_loc_long,ship_from_loc_coords,rec_ship_from_loc_coords,dest_coords,actual_distance_shipped,recommended_distance_shipped,eventTimeDate,actual_eventDate,actual_SHIP_NODE,act_loc_qty_day,act_loc_order_qty_day
0,2018111614203598721255,2018-11-16 20:20:36.833,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313,BBY_2600,38.06069100,-85.72090400,NaN,NaN,NaN,NaN,BBY_2600,38.06069100,-85.72090400,"(38.06069100, -85.72090400)","(38.06069100, -85.72090400)","(nan, nan)",NaN,NaN,2018-11-16,2018-11-16,BBY_2600,3366099,1310
1,2018111413520911168760,2018-11-14 19:52:11.261,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17,BBY_1458,29.04414900,-95.45613100,NaN,NaN,NaN,NaN,BBY_1458,29.04414900,-95.45613100,"(29.04414900, -95.45613100)","(29.04414900, -95.45613100)","(nan, nan)",NaN,NaN,2018-11-14,2018-11-14,BBY_1458,273,16
2,2018112005325987886463,2018-11-20 11:33:02.200,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9,BBY_1475,30.47299000,-81.64357000,BBY01-805582422630,32507,30.339837,-87.373783,BBY_1475,30.47299000,-81.64357000,"(30.47299000, -81.64357000)","(30.47299000, -81.64357000)","(30.339837, -87.37378299999999)",342.225758,342.225758,2018-11-20,2018-11-20,BBY_1475,76,8
3,2018112015160759573686,2018-11-20 21:16:11.278,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,02324,41.972387,-70.978778,BBY_519,33.72742100,-84.75784300,"(33.72742100, -84.75784300)","(33.72742100, -84.75784300)","(41.972387, -70.978778)",942.012832,942.012832,2018-11-20,2018-11-20,BBY_519,123,12
4,2018112015160759573686,2018-11-20 21:16:11.278,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,30101,34.034515,-84.707349,BBY_519,33.72742100,-84.75784300,"

In [32]:
expected_tp = pd.read_csv("./Data/data_expected_inventory_throughput_by_date_1-27-2019.txt", 
                         sep = '\t',
                         dtype = str
                        )
expected_tp.columns = ['expected_eventDate', 'expected_SHIP_NODE', 'exp_loc_qty_day', 'exp_loc_order_qty_day']
expected_tp.head(10)

,expected_eventDate,expected_SHIP_NODE,exp_loc_qty_day,exp_loc_order_qty_day
0,2018-11-11,BBY_10,48,9
1,2018-11-11,BBY_1002,66,15
2,2018-11-11,BBY_1003,35,12
3,2018-11-11,BBY_1004,17,8
4,2018-11-11,BBY_1008,14,13
5,2018-11-11,BBY_101,56,10
6,2018-11-11,BBY_1010,33,21
7,2018-11-11,BBY_1011,18,7
8,2018-11-11,BBY_1012,33,13
9,2018-11-11,BBY_1013,184,23


In [33]:
data_df = data_df.merge(expected_tp,
                        left_on = ['loc_id', 'eventTimeDate'],
                        right_on = ['expected_SHIP_NODE', 'expected_eventDate'],
                        how = 'left'
                       )
data_df.head(10)

,order_line_key,eventTime,inhand_date,sku,orderDate,order_no,order_line_type,zip_code,category,order_qty,order_los,loc_id,loc_type_desc,loc_cost,qty,loc_meet_in_hand,loc_rank_score,loc_shipping_method,loc_over_capacity,loc_ship_zone,loc_tnt,loc_type_code,loc_cut_off_time,loc_air_cut_off_time,CURR_LINE_STATUS,ORIG_CUST_PROMISE_DT,CURR_CUST_PROMISE_DT,SHIP_NODE,CARRIER,TRACKING_NUMBER,UPS_SVC_CDE,SHIP_DELAY_EMAIL_FLAG,EXTN_CARR_NM,actual_ship_method_ups,recommended_ship_method_ups,orderYMD,days_diff_planned,day_of_week,week_of_year,sec_diff_eventTM_orderDT,loc_qty_day,loc_order_qty_day,ship_from_loc_id,ship_from_loc_lat,ship_from_loc_long,dest_order_number,dest_zip_code,dest_lat,dest_long,rec_ship_from_loc_id,rec_ship_from_loc_lat,rec_ship_from_loc_long,ship_from_loc_coords,rec_ship_from_loc_coords,dest_coords,actual_distance_shipped,recommended_distance_shipped,eventTimeDate,actual_eventDate,actual_SHIP_NODE,act_loc_qty_day,act_loc_order_qty_day,expected_eventDate,expected_SHIP_NODE,exp_loc_qty_day,exp_loc_order_qty_day
0,2018111614203598721255,2018-11-16 20:20:36.833,2018-11-27 06:00:00 UTC,6151323,2018-11-16T20:20:34Z,BBYTX-805581185428,NIB,77494,BESTBUY,1,1,BBY_2600,STORE,2068873610176,461,1921958822953,1751785385,20020404405,80081617623,5005,15015303,MS,63000,63000,Return Received,2018-11-27,2018-11-27,BBY_2600,UPS,1ZE0A067YW66137784,YW,N,GRND,ground,ground,2018-11-16,11,Friday,46,2.83299994468689,3368232,1313,BBY_2600,38.06069100,-85.72090400,NaN,NaN,NaN,NaN,BBY_2600,38.06069100,-85.72090400,"(38.06069100, -85.72090400)","(38.06069100, -85.72090400)","(nan, nan)",NaN,NaN,2018-11-16,2018-11-16,BBY_2600,3366099,1310,2018-11-16,BBY_2600,3366099,1310
1,2018111413520911168760,2018-11-14 19:52:11.261,2018-11-15 06:00:00 UTC,5856129,2018-11-14T19:52:09Z,BBYTX-805580751818,NIB,77381,BESTBUY,1,7,BBY_1458,STORE,2068864147991,2,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,53100,53100,Shipped and Invoiced,2018-11-15,2018-11-15,BBY_1458,UPS,1ZR26W660325821086,03,N,UPSG,ground,ground,2018-11-14,1,Wednesday,46,2.26100015640259,277,17,BBY_1458,29.04414900,-95.45613100,NaN,NaN,NaN,NaN,BBY_1458,29.04414900,-95.45613100,"(29.04414900, -95.45613100)","(29.04414900, -95.45613100)","(nan, nan)",NaN,NaN,2018-11-14,2018-11-14,BBY_1458,273,16,2018-11-14,BBY_1458,277,17
2,2018112005325987886463,2018-11-20 11:33:02.200,2018-11-26 06:00:00 UTC,5349900,2018-11-20T11:32:57Z,BBY01-805582422630,NIB,32507,BESTBUY,1,4,BBY_1475,STORE,2068864049891,2,1921958822953,1751785385,20020404405,80081617623,4004,5005101,SDS,49500,55800,Shipped and Invoiced,2018-11-26,2018-11-26,BBY_1475,UPS,1ZR3A911YW57403208,YW,N,GRND,ground,ground,2018-11-20,6,Tuesday,47,5.20000004768372,19,9,BBY_1475,30.47299000,-81.64357000,BBY01-805582422630,32507,30.339837,-87.373783,BBY_1475,30.47299000,-81.64357000,"(30.47299000, -81.64357000)","(30.47299000, -81.64357000)","(30.339837, -87.37378299999999)",342.225758,342.225758,2018-11-20,2018-11-20,BBY_1475,76,8,2018-11-20,BBY_1475,13,7
3,2018112015160759573686,2018-11-20 21:16:11.278,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y40334073778,03,N,UPSG,ground,ground,2018-11-20,7,Tuesday,47,5.27799987792969,128,14,BBY_519,33.72742100,-84.75784300,BBY01-805582713455,02324,41.972387,-70.978778,BBY_519,33.72742100,-84.75784300,"(33.72742100, -84.75784300)","(33.72742100, -84.75784300)","(41.972387, -70.978778)",942.012832,942.012832,2018-11-20,2018-11-20,BBY_519,123,12,2018-11-20,BBY_519,128,14
4,2018112015160759573686,2018-11-20 21:16:11.278,2018-11-27 06:00:00 UTC,5873917,2018-11-20T21:16:06Z,BBY01-805582713455,NIB,30101,BESTBUY,2,4,BBY_519,STORE,2068864047854,37,1921958822953,1751785385,20020404405,80081617623,2002,5005101,MS,46800,45900,Shipped and Invoiced,2018-11-27,2018-11-27,BBY_519,UPS,1ZF701Y4033407377

In [34]:
data_df.to_csv('./Data/fully_joined_model_features.txt', sep = '\t')

In [6]:
import geopy.distance

ModuleNotFoundError: No module named 'geopy'

In [115]:
df = pd.read_excel("./Data/County_Rural_Lookup.xlsx", header = 3, usecols = 7)
df.head()

,2015 GEOID,State,2015 Geography Name,Note,2010 Census Total Population,2010 Census Urban Population,2010 Census Rural Population,2010 Census Percent Rural
0,01001,AL,"Autauga County, Alabama",NaN,54571.0,31650.0,22921.0,42.002162
1,01003,AL,"Baldwin County, Alabama",NaN,182265.0,105205.0,77060.0,42.279099
2,01005,AL,"Barbour County, Alabama",NaN,27457.0,8844.0,18613.0,67.789635
3,01007,AL,"Bibb County, Alabama",NaN,22915.0,7252.0,15663.0,68.352607
4,01009,AL,"Blount County, Alabama",NaN,57322.0,5760.0,51562.0,89.951502


In [123]:
df['urbanicity_flag'] = df.iloc[:,7].apply(lambda x: 'MU' if x < 50 else ('MR' if x < 100 else 'R'))
df.head()

,2015 GEOID,State,2015 Geography Name,Note,2010 Census Total Population,2010 Census Urban Population,2010 Census Rural Population,2010 Census Percent Rural,urbanicity_flag
0,01001,AL,"Autauga County, Alabama",NaN,54571.0,31650.0,22921.0,42.002162,MU
1,01003,AL,"Baldwin County, Alabama",NaN,182265.0,105205.0,77060.0,42.279099,MU
2,01005,AL,"Barbour County, Alabama",NaN,27457.0,8844.0,18613.0,67.789635,MR
3,01007,AL,"Bibb County, Alabama",NaN,22915.0,7252.0,15663.0,68.352607,MR
4,01009,AL,"Blount County, Alabama",NaN,57322.0,5760.0,51562.0,89.951502,MR


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3148 entries, 0 to 3147
Data columns (total 8 columns):
2015 GEOID                      3146 non-null object
State                           3142 non-null object
2015 Geography Name             3142 non-null object
Note                            56 non-null float64
2010 Census Total Population    3142 non-null float64
2010 Census Urban Population    3142 non-null float64
2010 Census Rural Population    3142 non-null float64
2010 Census 
Percent Rural      3142 non-null float64
dtypes: float64(5), object(3)
memory usage: 196.8+ KB


In [103]:
# Choice of Vintage and Benchmark done as Urban/Rural Flag calculated by county
params = {
    "benchmark": 9,
    "vintage" : 910,
    "format" : "json",
    "address" : "1500 REMINGTON BLVD, BOLINGBROOK, IL, 60490"
}

In [106]:
res['result']['addressMatches']

[{'matchedAddress': '1500 Remington Blvd, BOLINGBROOK, IL, 60490',
  'coordinates': {'x': -88.12594, 'y': 41.6624},
  'tigerLine': {'tigerLineId': '111624107', 'side': 'R'},
  'addressComponents': {'fromAddress': '1500',
   'toAddress': '1698',
   'preQualifier': '',
   'preDirection': '',
   'preType': '',
   'streetName': 'Remington',
   'suffixType': 'Blvd',
   'suffixDirection': '',
   'suffixQualifier': '',
   'city': 'BOLINGBROOK',
   'state': 'IL',
   'zip': '60490'},
  'geographies': {'States': [{'STATENS': '01779784',
     'POP100': 12830632,
     'GEOID': '17',
     'CENTLAT': '+40.1006007',
     'AREAWATER': 6202038080,
     'STATE': '17',
     'BASENAME': 'Illinois',
     'STUSAB': 'IL',
     'OID': 27490100360203,
     'LSADC': '00',
     'FUNCSTAT': 'A',
     'INTPTLAT': '+40.1028754',
     'DIVISION': '3',
     'NAME': 'Illinois',
     'REGION': '2',
     'OBJECTID': 40,
     'CENTLON': '-089.1500807',
     'AREALAND': 143793362385,
     'INTPTLON': '-089.1526108',
     

In [110]:
test_result['States'][0]

{'STATENS': '01779784',
 'POP100': 12830632,
 'GEOID': '17',
 'CENTLAT': '+40.1006007',
 'AREAWATER': 6202038080,
 'STATE': '17',
 'BASENAME': 'Illinois',
 'STUSAB': 'IL',
 'OID': 27490100360203,
 'LSADC': '00',
 'FUNCSTAT': 'A',
 'INTPTLAT': '+40.1028754',
 'DIVISION': '3',
 'NAME': 'Illinois',
 'REGION': '2',
 'OBJECTID': 40,
 'CENTLON': '-089.1500807',
 'AREALAND': 143793362385,
 'INTPTLON': '-089.1526108',
 'HU100': 5296715,
 'MTFCC': 'G4000',
 'UR': ''}

In [111]:
test_result['Counties'][0]

{'POP100': 677560,
 'GEOID': '17197',
 'CENTLAT': '+41.4449657',
 'AREAWATER': 31904495,
 'STATE': '17',
 'BASENAME': 'Will',
 'OID': 27590164305374,
 'LSADC': '06',
 'FUNCSTAT': 'A',
 'INTPTLAT': '+41.4484742',
 'NAME': 'Will County',
 'OBJECTID': 99,
 'CENTLON': '-087.9784737',
 'COUNTYCC': 'H1',
 'COUNTYNS': '01785190',
 'AREALAND': 2167580722,
 'INTPTLON': '-087.9784564',
 'HU100': 237501,
 'MTFCC': 'G4020',
 'UR': '',
 'COUNTY': '197'}

In [112]:
test_result['Census Tracts'][0]

{'POP100': 9463,
 'GEOID': '17197880121',
 'CENTLAT': '+41.6727954',
 'AREAWATER': 0,
 'STATE': '17',
 'BASENAME': '8801.21',
 'OID': 207903711048263,
 'LSADC': 'CT',
 'FUNCSTAT': 'S',
 'INTPTLAT': '+41.6727954',
 'NAME': 'Census Tract 8801.21',
 'OBJECTID': 21203,
 'TRACT': '880121',
 'CENTLON': '-088.1197366',
 'HU100': 2707,
 'AREALAND': 14901942,
 'INTPTLON': '-088.1197366',
 'MTFCC': 'G5020',
 'UR': '',
 'COUNTY': '197'}

In [113]:
test_result['Census Blocks'][0]

{'SUFFIX': '',
 'POP100': 0,
 'GEOID': '171978801213048',
 'CENTLAT': '+41.6632011',
 'BLOCK': '3048',
 'AREAWATER': 0,
 'STATE': '17',
 'BASENAME': '3048',
 'OID': 210404014428373,
 'LSADC': 'BK',
 'FUNCSTAT': 'S',
 'INTPTLAT': '+41.6632011',
 'NAME': 'Block 3048',
 'OBJECTID': 3116698,
 'TRACT': '880121',
 'CENTLON': '-088.1312783',
 'BLKGRP': '3',
 'AREALAND': 501734,
 'HU100': 0,
 'INTPTLON': '-088.1312783',
 'MTFCC': 'G5040',
 'LWBLKTYP': 'L',
 'UR': '',
 'COUNTY': '197'}